In [6]:
import pandas as pd
import numpy as np
import time
import datetime

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
import scipy as sp
import re
import gensim
import logging
import seaborn as sns
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
%matplotlib inline
#import xgboost

train = pd.read_csv("data/features/features.csv",
                    header=0,delimiter=",",quotechar='"',error_bad_lines=False)

print("unfiltered data, num samples: %i with num features: %i" %(train.shape[0],train.shape[1]))

train.fillna(' ',inplace=True)
train.dropna(inplace=True)
print("filtered data with samples: %i" %(train.shape[0]))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
unfiltered data, num samples: 45885 with num features: 22
filtered data with samples: 45885


In [7]:
abstracts = train["abstract"].values
idxthere = np.nonzero(abstracts != ' ')[0]
useful = train["useful"]
abstracts = abstracts[idxthere]
useful = useful[idxthere]

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import time
from nltk.corpus import stopwords

start = time.time()

documents = abstracts
reRemoved = [re.sub(r'[^a-z ]', '', s.lower()) for s in documents]
vect = CountVectorizer(max_df=0.6,min_df=10,stop_words=stopwords.words("english"))
bow = vect.fit_transform(reRemoved)

In [11]:
import xgboost as xgb
# read in data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(bow, useful, test_size=0.2, random_state=42)

model = xgb.XGBClassifier(max_depth=6,n_estimators=200)
model.fit(X_train, y_train)


/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [26]:
preds = model.predict_proba(X_test)[:,1]
print(preds.shape)
print(y_test.shape)
#np.mean(np.power(preds - y_test,2))
sk.metrics.roc_auc_score(y_test,preds)

(8574,)
(8574,)


0.8048125206870721